##### Master Degree in Computer Science and Data Science for Economics

# Statistical Language Models

### Alfio Ferrara

Given a corpus $D$ of size $N$, we can compute the probability of a token $w \in D$ as:

$$
P(w) = \frac{count(w)}{\sum\limits_{i=0}^{N} count(w_i)}
$$

Using this simple statistics, we can sample a word from the corpus according to its probability $P(w)$. If we use this to generate a text $d = w_1, w_2, \dots, w_{n-1}, w_n$ we have:

$$
P(w_1, w_2, \dots, w_{n-1}, w_n) = P(w_1)P(w_2) \dots P(w_{n-1})P(w_n) = \prod\limits_{i=1}^{n} P(w_i)
$$

Howewer, this is not realistic. A better way to model this process is to choose words by taking into account the words we generated before, by sampling the $i$th word with a probabiity that is conditioned by the previous $i-1$ words.

Thus, applying the chain rule:

$$
P(w_1, w_2, \dots, w_{n-1}, w_n) = P(w_1) P(w_2 \mid w_1) \dots P(w_{n-1} \mid w_1 \dots w_{n-2}) P(w_n \mid w_1 \dots w_{n-1}) = \prod\limits_{i=1}^{n} P(w_i \mid w_1 \dots w_{i-1})
$$


Now, indexing long sequences as required by this method is unfeasible for a couple of good reasons:
1. it's memory consuming and computationally intractable
2. The logest the sequences the fewer the chances to observe such sequences a sufficient number of times

So, we can apply a Markov approximation by taking into account only subsequences of lenght $k$:

$$
P(w_1, w_2, \dots, w_{n-1}, w_n) = \prod\limits_{i=1}^{n} P(w_i \mid w_{i-k} \dots w_{i-1})
$$

In [ ]:
from nlp.langmodel import MarkovLM
import pymongo

## Get a corpus of texts

In [ ]:
db = pymongo.MongoClient()['cousine']
recipes = db['foodcom']

In [ ]:
def create_corpus(query:  dict = {}, numdocs: int = 3000):
    corpus = []
    for recipe in recipes.find(query).limit(numdocs):
        for sentence in recipe['steps']:
            corpus.append(sentence)
    return corpus

In [ ]:
numdocs = 3000
corpus = create_corpus(query={}, numdocs=numdocs)
print(f"Corpus size: {len(corpus)}")
for text in corpus[:4]:
    print(text)

Corpus size: 20262
I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.
Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.
Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.
Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.


### N-order markov assumption (n-gram Markov Language Model)
We assume that a word (to predict) depends only on the previous n words.
$$P(W_1...W_k) = \prod_{i=1}^{k}  P(W_k | W_1,..., W_k-1)  $$

* k=2:
specifies a 2nd-order Markov model, also known as a bigram model.
This model predicts the probability of the next word based on the single preceding word.
The model learns the probabilities of sequences of two words (bigrams).

* k=4: specifies a 4th-order Markov model, also known as a four-gram model.
This model predicts the probability of the next word based on the three preceding words.
The model learns the probabilities of sequences of four words (four-grams).

In [ ]:
tokenizer = "bert-base-uncased"
brlm = MarkovLM(k=2, tokenizer_model=tokenizer) #Bigram-based Markov LM
frlm = MarkovLM(k=4, tokenizer_model=tokenizer) #Four-gram-based Markov LM

In [ ]:
brlm.train(corpus=corpus)
frlm.train(corpus=corpus)

100%|██████████| 20262/20262 [00:02<00:00, 7508.11it/s]


## Text generation
 MarkovLM().generate() would use the trained n-gram Markov Language Model to produce a new sequence of text. This process is often called text generation or sampling.  
 MarkovLM().generate()  (or any similar text generation function) will likely not give the same result each time. It will produce a new, different sequence of text with each run.

In [ ]:
brlm.generate()

['[#S]',
 'pre',
 '##hea',
 '##t',
 'slowly',
 'add',
 'the',
 'cottage',
 'cheese',
 'and',
 'then',
 'add',
 'the',
 'oven',
 'for',
 '30',
 'minutes',
 '.',
 '[#E]']

In [ ]:
print("2gram: ", " ".join(brlm.generate()).replace(" ##", ""))
print("4gram: ", " ".join(frlm.generate()).replace(" ##", ""))

2gram:  [#S] with walnuts in the beans and bring to cover and 1 minute . [#E]
4gram:  [#S] [#S] [#S] in a heavy skillet , saute ham with onions and dot with butter . [#E]


## Text classification
The primary purpose of MarkovLM.log_prob() is to compute the log-probability of a given sequence of tokens occurring according to the model's learned transition probabilities.
It's a method for calculating the probability of a given sequence of text under a trained Markov Language Model. While this probability score can be used for a form of text classification, it's not a direct or robust method.
Note that the MarkovLM() should be trained separately for each class.

In [ ]:
italian_q = {'search_terms': 'italian'}
chinese_q = {'search_terms': 'chinese'}
numdocs = 3000
italian_corpus = create_corpus(query=italian_q, numdocs=numdocs)
chinese_corpus = create_corpus(query=chinese_q, numdocs=numdocs)
print(f"Italian: {len(italian_corpus)}")
print(f"Chinese: {len(chinese_corpus)}")

Italian: 30052
Chinese: 24134


In [ ]:
it = MarkovLM(k=4, tokenizer_model=tokenizer)
ch = MarkovLM(k=4, tokenizer_model=tokenizer)

In [ ]:
it.train(corpus=italian_corpus)
ch.train(corpus=chinese_corpus)

100%|██████████| 24134/24134 [00:03<00:00, 6844.65it/s]


In [ ]:
italian_sentence = italian_corpus[6]
chinese_sentence = chinese_corpus[6]

print(f"Italian sentence: {italian_sentence}")
print(f"Italian: {it.log_prob(italian_sentence)}")
print(f"Chinese: {ch.log_prob(italian_sentence)}")
print("========")
print(f"Chinese sentence: {chinese_sentence}")
print(f"Italian: {it.log_prob(chinese_sentence)}")
print(f"Chinese: {ch.log_prob(chinese_sentence)}")
#a log-probability of -23.09 indicates a much less likely sequence than a log-probability of -21.35

Italian sentence: Place ravioli on a large baking sheet sprinkled with cornstarch.
Italian: -23.0900057470743
Chinese: -21.352223087931076
Chinese sentence: Heat enough oil in a frying pan over medium heat to shallow fry.
Italian: -24.643378642963395
Chinese: -22.686133859693925


## Combine languages

In [ ]:
import copy

In [ ]:
mix = copy.deepcopy(it)

In [ ]:
mix.train(chinese_corpus)

100%|██████████| 24134/24134 [00:03<00:00, 7124.59it/s]


In [ ]:
print("Mix: ", " ".join(mix.generate()).replace(" ##", ""))

Mix:  [#S] [#S] [#S] add squash and sage ; saute in extra virgin olive oil again . close the lid , turn up the heat to medium . pour water into a small cigar shape . make sure rice takes up soya sauce and 1 / 4 teaspoon , scoop out any seeds and then press down on it slightly since eggplant . slice eggplant on a folded brown paper bag or on paper towels . [#E]
